# Generic Shooting

In [16]:
import numpy as np
from scipy.optimize import bisect
from scipy.integrate import odeint

# temporary
# import schrodinger specific stuff
from .schrodinger import *

def find_energies(V, n=3):
    """Find the energies allowed by the TISE for a particular potential.
    
    Parameters:
    -----------
    V : function
        the potential
    n : integer, optional
        how many energies to find; defaults to 3
        
    Returns:
    --------
    energies : list
               energy values
    """
    
    if not callable(V):
        raise TypeError("{} is not callable".format(V))
    energies = []
    bracket_found = False
    starting_value = 0.01
    e_step = 0.3
    
    # prime the pump
    prev_energy = starting_value
    endpoint = turning_point(V, prev_energy) + 8
    prev_bc = bc(prev_energy, V, endpoint)
    
    for idx in range(n):
        # try energies until we find two values that bracket an eigenvalue
        while not bracket_found:
            current_energy = prev_energy + e_step
            endpoint = turning_point(V, current_energy) + 8
            current_bc = bc(current_energy, V, endpoint)
            if np.sign(current_bc) != np.sign(prev_bc):
                bracket_found = True
            else:
                prev_energy = current_energy
                prev_bc = current_bc
        # use bisect to get the actual value
        energies.append(bisect(bc, prev_energy, current_energy, args=(V,endpoint,)))
        
        # set up for the next energy
        # populate prev_energy
        prev_energy = current_energy
        prev_bc = current_bc
        # we no longer have a bracket
        bracket_found = False
    return energies

def bc(E, V, endpoint=5):
    """Find the boundary value for a given energy.
    
    Parameters:
    -----------
    E : number
        The energy to test
    V : function
        The potential function
    endpoint : number, optional
               the endpoint for the integration interval.
    Returns:
    --------
    psi(endpoint) : number
             The value of psi at x=endpoint. 
    """
    domain = np.linspace(-endpoint,endpoint,1000)
    y0 = (0.0, 1.0)
        
    result = odeint(schro_rhs, y0, domain, args=(E,V,))
    return result[-1, 0]

ImportError: attempted relative import with no known parent package

In [18]:
%save "shooting/shooting" 16

The following commands were written to file `shooting/shooting.py`:
import numpy as np
from scipy.optimize import bisect
from scipy.integrate import odeint

# temporary
# import schrodinger specific stuff
from .schrodinger import *

def find_energies(V, n=3):
    """Find the energies allowed by the TISE for a particular potential.
    
    Parameters:
    -----------
    V : function
        the potential
    n : integer, optional
        how many energies to find; defaults to 3
        
    Returns:
    --------
    energies : list
               energy values
    """
    
    if not callable(V):
        raise TypeError("{} is not callable".format(V))
    energies = []
    bracket_found = False
    starting_value = 0.01
    e_step = 0.3
    
    # prime the pump
    prev_energy = starting_value
    endpoint = turning_point(V, prev_energy) + 8
    prev_bc = bc(prev_energy, V, endpoint)
    
    for idx in range(n):
        # try energies until we find two values that bracket an eigenva

# Schrodinger-specific

In [1]:
import numpy as np

def schro_rhs(y, x, E, V, m=1.0, hbar=1.0):
    """RHS for the time independent schrodinger equation.
    
    Parameters:
    -----------
    y : iterable of floats
        contains $\psi(x)$ and $\psi'(x)$
    x : float
        the position at which we're evaluating the equation
    E : float
        Energy of the quantum state
    V : function
        function for the potential
    m : float, optional
        mass; defaults to 1
    hbar : float, optional
           Planck's constant/2pi; defaults to 1
    
    Returns:
    --------
    float : the right hand side of the time independent schrodinger
            equation for the time independent Schrodinger equation:
            $$\frac{d^2\psi}{dx^2} = -\frac{2m}{\hbar^2}
            \left(E - V(x) \right)\psi.$$
    """
    psi, psiprime = y
    psidoubleprime = -(2*m/hbar**2)*(E-V(x))*psi
    
    return np.array([psiprime, psidoubleprime])

def turning_point(V, E):
    """find the classical turning point for the potential and energy"""
    
    # this works for SHO; won't for other potentials
    A = V(1)
    return np.sqrt(E/A)

def harmonic(x, m=1, omega=1):
    """Potential for the harmonic oscillator"""
    
    return m*omega**2*x**2/2

In [12]:
%save "shooting/schrodinger" 1

The following commands were written to file `shooting/schrodinger.py`:
import numpy as np

def schro_rhs(y, x, E, V, m=1.0, hbar=1.0):
    """RHS for the time independent schrodinger equation.
    
    Parameters:
    -----------
    y : iterable of floats
        contains $\psi(x)$ and $\psi'(x)$
    x : float
        the position at which we're evaluating the equation
    E : float
        Energy of the quantum state
    V : function
        function for the potential
    m : float, optional
        mass; defaults to 1
    hbar : float, optional
           Planck's constant/2pi; defaults to 1
    
    Returns:
    --------
    float : the right hand side of the time independent schrodinger
            equation for the time independent Schrodinger equation:
            $$\frac{d^2\psi}{dx^2} = -\frac{2m}{\hbar^2}
            \left(E - V(x) \right)\psi.$$
    """
    psi, psiprime = y
    psidoubleprime = -(2*m/hbar**2)*(E-V(x))*psi
    
    return np.array([psiprime, psidoubleprime])


# Tests

In [19]:
import numpy as np
from numpy.testing import assert_allclose

# import the function(s) to test
from .shooting import find_energies

def test_find_energies_sho(n=3):
    """Test that find_energies gives correct results for the harmonic oscillator."""
    
    def sho_test(x):
        """Potential for the harmonic oscillator, with m=1, omega=1"""    
        return x**2/2

    desired_energies = np.array([i+0.5 for i in range(n)])
    assert_allclose(find_energies(sho_test, n), desired_energies)

ImportError: attempted relative import with no known parent package

In [20]:
%save "shooting/tests" 19

The following commands were written to file `shooting/tests.py`:
import numpy as np
from numpy.testing import assert_allclose

# import the function(s) to test
from .shooting import find_energies

def test_find_energies_sho(n=3):
    """Test that find_energies gives correct results for the harmonic oscillator."""
    
    def sho_test(x):
        """Potential for the harmonic oscillator, with m=1, omega=1"""    
        return x**2/2

    desired_energies = np.array([i+0.5 for i in range(n)])
    assert_allclose(find_energies(sho_test, n), desired_energies)


# Plots

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# import the shooting stuff
from .shooting import find_energies
from .schrodinger import schro_rhs

def plot_energy_diagram(V, n=3):
    """Plot an energy diagram for a potential well."""
    
    domain = np.linspace(-5, 5, 1000)
    halfdomain = np.linspace(0, 5, 500)
    potential = V(domain)
    minx = np.min(domain)
    maxx = np.max(domain)
    
    fig, ax = plt.subplots(1)
    fig.set_figwidth(10)
    fig.set_figheight(6)
    
    ax.plot(domain, potential)
    ax.set_axis_off()
    
    # plot energies and eigenfunctions
    energies = find_energies(V, n)
    even = True
    for energy in energies:
        ax.plot([minx, maxx], [energy, energy], ":", color="gray")
        y0 = (0.0, 1.0)
        result = odeint(schro_rhs, y0, domain, args=(energy, V))
        eigenfunction = 0.3*(result[:,0]) + energy
        ax.plot(domain, eigenfunction)
        ax.text(4.5, energy,"$E={:5.2f}$".format(energy),  verticalalignment="bottom")
        even = not even
    deltae = energies[-1] - energies[-2]    
    ax.set_ylim(-0.1, energies[-1] + deltae)

ImportError: attempted relative import with no known parent package

In [22]:
%save shooting/plots 21

The following commands were written to file `shooting/plots.py`:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# import the shooting stuff
from .shooting import find_energies
from .schrodinger import schro_rhs

def plot_energy_diagram(V, n=3):
    """Plot an energy diagram for a potential well."""
    
    domain = np.linspace(-5, 5, 1000)
    halfdomain = np.linspace(0, 5, 500)
    potential = V(domain)
    minx = np.min(domain)
    maxx = np.max(domain)
    
    fig, ax = plt.subplots(1)
    fig.set_figwidth(10)
    fig.set_figheight(6)
    
    ax.plot(domain, potential)
    ax.set_axis_off()
    
    # plot energies and eigenfunctions
    energies = find_energies(V, n)
    even = True
    for energy in energies:
        ax.plot([minx, maxx], [energy, energy], ":", color="gray")
        y0 = (0.0, 1.0)
        result = odeint(schro_rhs, y0, domain, args=(energy, V))
        eigenfunction = 0.3*(result[:,0]) + energy
        ax.plot(domain, ei

# turning this into a functional package

In [11]:
from .shooting import *
from .schrodinger import *
from .plots import *

ImportError: attempted relative import with no known parent package

In [12]:
%save shooting/__init__.py %

File `shooting/__init__.py` exists. Overwrite (y/[N])?  y
'%' was not found in history, as a file, url, nor in the user namespace.


In [15]:
_ih[-3]

'type(_ih)'

In [1]:
import shooting

In [2]:
help(shooting.find_energies)

Help on function find_energies in module shooting.shooting:

find_energies(V, n=3)
    Find the energies allowed by the TISE for a particular potential.
    
    Parameters:
    -----------
    V : function
        the potential
    n : integer, optional
        how many energies to find; defaults to 3
        
    Returns:
    --------
    energies : list
               energy values



In [3]:
shooting.find_energies(shooting.harmonic)

[0.5000000045123304, 1.50000000277556, 2.4999999969919005]

In [6]:
%history -l1

%history


In [5]:
%history

import shooting
help(shooting.find_energies)
shooting.find_energies(shooting.harmonic)
%history -1
%history


In [2]:
import shooting